In [ ]:
# ------------------- Import Libraries -------------------
import tkinter as tk
from tkinter import ttk, simpledialog, messagebox
import json
import os

# ------------------- File for Storing Rates -------------------
RATES_FILE = "exchange_rates.json"

# ------------------- Load Exchange Rates -------------------
def load_rates():
    """Load rates from file if exists, otherwise use defaults"""
    if os.path.exists(RATES_FILE):
        with open(RATES_FILE, "r") as f:
            return json.load(f)
    else:
        # Default rates if file doesn't exist
        return {
            "USD": 1.0,
            "EUR": 0.93,
            "GBP": 0.80,
            "NGN": 1530.00,
            "JPY": 149.00
        }

# ------------------- Save Exchange Rates -------------------
def save_rates():
    """Save current rates to file"""
    with open(RATES_FILE, "w") as f:
        json.dump(exchange_rates, f)

# ------------------- Load rates into dictionary -------------------
exchange_rates = load_rates()

# ------------------- Conversion Function -------------------
def convert_currency():
    """Perform currency conversion based on user input"""
    try:
        amount = float(amount_entry.get())
        from_currency = from_currency_cb.get()
        to_currency = to_currency_cb.get()

        # Convert: From → USD → To
        usd_amount = amount / exchange_rates[from_currency]
        converted_amount = usd_amount * exchange_rates[to_currency]

        result_label.config(
            text=f"{amount} {from_currency} = {round(converted_amount, 2)} {to_currency}"
        )
    except ValueError:
        result_label.config(text="Please enter a valid number!")

# ------------------- Swap Function -------------------
def swap_currencies():
    """Swap 'From' and 'To' currencies automatically"""
    from_curr = from_currency_cb.get()
    to_curr = to_currency_cb.get()
    from_currency_cb.set(to_curr)
    to_currency_cb.set(from_curr)

# ------------------- Update Existing Rates -------------------
def update_rate():
    """Allow user to update exchange rate and save it permanently"""
    currency = simpledialog.askstring("Update Rate", "Enter currency code (e.g., USD, EUR, NGN):")
    
    if currency and currency.upper() in exchange_rates:
        try:
            new_rate = float(simpledialog.askstring("Update Rate", f"Enter new rate for {currency.upper()} (1 USD = ? {currency.upper()}):"))
            
            # Update dictionary
            exchange_rates[currency.upper()] = new_rate
            
            # Save to file
            save_rates()
            
            messagebox.showinfo("Success", f"Exchange rate for {currency.upper()} updated to {new_rate}")
        
        except ValueError:
            messagebox.showerror("Error", "Please enter a valid number!")
    else:
        messagebox.showerror("Error", "Currency not found!")

# ------------------- Add New Currency -------------------
def add_currency():
    """Allow user to add a completely new currency"""
    currency = simpledialog.askstring("Add Currency", "Enter NEW currency code (e.g., CAD, INR):")
    
    if currency:
        currency = currency.upper()
        if currency in exchange_rates:
            messagebox.showwarning("Warning", f"{currency} already exists!")
            return
        
        try:
            rate = float(simpledialog.askstring("Add Currency", f"Enter exchange rate for {currency} (1 USD = ? {currency}):"))
            
            # Add new currency
            exchange_rates[currency] = rate
            
            # Save to file
            save_rates()
            
            # Update dropdown menus with new currency
            from_currency_cb["values"] = list(exchange_rates.keys())
            to_currency_cb["values"] = list(exchange_rates.keys())
            
            messagebox.showinfo("Success", f"Currency {currency} added with rate {rate}")
        
        except ValueError:
            messagebox.showerror("Error", "Please enter a valid number!")

# ------------------- GUI Setup -------------------
root = tk.Tk()
root.title("Currency Converter (Offline with Saved Rates + Add Currency)")  
root.geometry("450x450")                    
root.config(bg="#f5f5f5")                   

# ------------------- Title -------------------
title_label = tk.Label(root, text="Currency Converter (Offline)", 
                       font=("Arial", 16, "bold"), bg="#f5f5f5")
title_label.pack(pady=10)

# ------------------- Amount -------------------
tk.Label(root, text="Amount:", font=("Arial", 12), bg="#f5f5f5").pack()
amount_entry = tk.Entry(root, font=("Arial", 12))
amount_entry.pack(pady=5)

# ------------------- From Currency -------------------
tk.Label(root, text="From Currency:", font=("Arial", 12), bg="#f5f5f5").pack()
from_currency_cb = ttk.Combobox(root, values=list(exchange_rates.keys()), font=("Arial", 12))
from_currency_cb.set("USD")
from_currency_cb.pack(pady=5)

# ------------------- To Currency -------------------
tk.Label(root, text="To Currency:", font=("Arial", 12), bg="#f5f5f5").pack()
to_currency_cb = ttk.Combobox(root, values=list(exchange_rates.keys()), font=("Arial", 12))
to_currency_cb.set("NGN")
to_currency_cb.pack(pady=5)

# ------------------- Buttons -------------------
convert_btn = tk.Button(root, text="Convert", command=convert_currency,
                        font=("Arial", 12, "bold"), bg="#4CAF50", fg="white")
convert_btn.pack(pady=10)

swap_btn = tk.Button(root, text="Swap Currencies", command=swap_currencies,
                     font=("Arial", 12, "bold"), bg="#2196F3", fg="white")
swap_btn.pack(pady=5)

update_btn = tk.Button(root, text="Update Exchange Rate", command=update_rate,
                       font=("Arial", 12, "bold"), bg="#FF9800", fg="white")
update_btn.pack(pady=5)

add_btn = tk.Button(root, text="Add New Currency", command=add_currency,
                    font=("Arial", 12, "bold"), bg="#9C27B0", fg="white")
add_btn.pack(pady=5)

# ------------------- Result -------------------
result_label = tk.Label(root, text="", font=("Arial", 14, "bold"),
                        bg="#f5f5f5", fg="blue")
result_label.pack(pady=20)

# ------------------- Run -------------------
root.mainloop()
